## GET FRAMES FOR VALIDATION TRAIN & TEST DATASET

In [8]:
import pandas as pd
import os
import json

In [9]:
output_prefix_path = "/data0/Results"
raw_input_prefix_path = "/data0/Data"

In [10]:
#input_file = "/data0/Data/ig_raw_frames/{}"
input_file = "{{}}/ig_raw_frames/{}"

#output_train_file = "/data0/Results/lift_classification/train_lift_frames/{}"
output_train_file = "{}/lift_classification/train_lift_frames.json"

#output_test_file = "/data0/Results/lift_classification/test_lift_frames/{}"
output_test_file = "{}/lift_classification/test_lift_frames.json"

#output_validation_file = "/data0/Results/lift_classification/train_lift_frames/{}"
output_validation_file = "{}/lift_classification/validation_lift_frames.json"

train_path = "/data0/Data/composite_jsons/data_splits/train.json"
test_path = "/data0/Data/composite_jsons/data_splits/test.json"
validation_path = "/data0/Data/composite_jsons/data_splits/validation.json"
composite_json_path = "/data0/Data/composite_jsons/{}"

In [11]:
train_data = pd.read_json(train_path)
test_data = pd.read_json(test_path)
validation_data = pd.read_json(validation_path)

In [12]:
train_data.head()

,composite_json,macro,micro
0,35538998_2056904291295716_6072990298055639040_...,deadlift,[conventional]
1,36675596_210677146256183_549651318840596073_n_...,deadlift,[conventional]
2,36408131_488686898237457_4752072826997964800_n...,benchpress,[regular]
3,36771650_1722039104569900_3105120230972588032_...,deadlift,[conventional]
4,36566401_200430427332741_4703183106893414400_n...,deadlift,[conventional]


In [13]:
def get_frames(dataset, outpath):
    data = {}
    for index, row in dataset.iterrows():
        lift_type = row['macro']
        file_name = row['composite_json']
        with open(composite_json_path.format(lift_type + "/" + file_name)) as j_file:
            json_data = json.load(j_file)
            
            raw_filename = json_data["metadata"]["filename"] # rawfile concanetaed with image.
            raw_folder = json_data["metadata"]["folder"]
            
            # raw_input_directory = "{}/ig_raw_frames/benchpress/xxxx_n"
            raw_input_directory = input_file.format(raw_folder + "/" + raw_filename)
            for image in os.listdir(raw_input_directory.format(raw_input_prefix_path)):
                idx = int(image.strip('.jpg'))
                if idx % 5 == 0:
                    data[raw_filename + "_" + image] = {"path": raw_input_directory + "/" + image,
                                                        "label": lift_type,
                                                        "filename": raw_filename}
                    
    if not os.path.exists(outpath.format(output_prefix_path)):
        with open(outpath.format(output_prefix_path), 'w') as outfile:
            json.dump(data, outfile, indent=4, separators=(',', ': '))
        print("Number of images/frames: {}".format(len(data.keys())))

In [14]:
# Get train frames
get_frames(train_data, output_train_file)

Number of images/frames: 1401110


In [15]:
# Get test frames
get_frames(test_data, output_test_file)

Number of images/frames: 298290


In [16]:
# Get validation frames
get_frames(validation_data, output_validation_file)

Number of images/frames: 197475
